In [26]:
from nltk.corpus import movie_reviews
movie_reviews.categories()
movie_reviews.fileids('neg')
movie_reviews.words(movie_reviews.fileids()[5])
documents = []
for category in movie_reviews.categories():
    for fileid in movie_reviews.fileids(category):
        documents.append((movie_reviews.words(fileid), category))
documents[0:5]
import random
random.shuffle(documents)
documents[0:5]
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.corpus import wordnet
def get_simple_pos(tag):
    
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
from nltk import pos_tag
w = "better"
pos_tag([w])
from nltk.corpus import stopwords
import string
stops = set(stopwords.words('english'))
punctuations = list(string.punctuation)
stops.update(punctuations)
stops, string.punctuation
def clean_review(words):
    output_words = []
    for w in words:
        if w.lower() not in stops:
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words
documents = [(clean_review(document), category) for document, category in documents]
training_documents = documents[0:1000]
testing_documents = documents[1000:]
all_words = []
for doc in training_documents:
    all_words += doc[0]
import nltk
freq = nltk.FreqDist(all_words)
common = freq.most_common(3000)
features = [i[0] for i in common]
def get_feature_dict(words):
    current_features = {}
    words_set = set(words)
    for w in features:
        current_features[w] = w in words_set
    return current_features
output = get_feature_dict(training_documents[0][0])
output
training_data = [(get_feature_dict(doc), category) for doc, category in training_documents]
testing_data = [(get_feature_dict(doc), category) for doc, category in training_documents]
from nltk import NaiveBayesClassifier
classfier = NaiveBayesClassifier.train(training_data)
nltk.classify.accuracy(classfier, testing_data)
import pickle

# Save the trained model
with open('naive_bayes_model.pkl', 'wb') as f:
    pickle.dump(classfier, f)

# Save the features list
with open('features.pkl', 'wb') as f:
    pickle.dump(features, f)

# Save the lemmatizer and stopwords (optional)
with open('lemmatizer.pkl', 'wb') as f:
    pickle.dump(lemmatizer, f)

with open('stops.pkl', 'wb') as f:
    pickle.dump(stops, f)

accuracy = nltk.classify.accuracy(classfier, testing_data)
print(f"Model accuracy: {accuracy}")


Model accuracy: 0.894
